SET UP

In [2]:
# Save the directory
directory =  '/content/drive/MyDrive/GenAI/OpenAI/OpenAI Project'

In [3]:
! pip install openai

In [4]:
from google.colab import userdata
openai_api_key = userdata.get('genai_course')

In [20]:
#load the libraries
from openai import OpenAI
import os
import base64
from IPython.display import Image, display, Markdown
import pandas as pd

In [6]:
#set the openai connection and model
Model = "gpt-4o"
client = OpenAI(api_key=openai_api_key)

In [57]:
# Define the system prompt
system_prompt = """
convert the meneu image to a structured excel sheet format following the provided template and instructions.
This assistence converts resturants and cafe menue data into a structured excel sheet that adheres to the specific template.
The template includes categories, subcategories, item names, prices, descriptiions a, and more, ensuring data consistency.
This assistent help useres to fill out each row correctly, following the detailed instructions provided.




Overview:
- Each row in the uploaded Excel spreadsheet represents a unique item categorized under a category or subcategory.
- Category and Subcategory names are repeated for items within the same subcategory.
- If an item is directly under a category, the subcategory columns should be left blank.
- Each item entry must have unique details, including names, prices, descriptions, and other relevant attributes.
- The uploaded menu content will be appended to the existing menu without deleting any current menu data.

Columns Guide:
Each column should follow the specified format:

CategoryTitlePt (Column A):
- Description: Category names in Portuguese
- Accepted Values: Text, max 256 characters
- Example: Bebidas

CategoryTitleEn (Column B) (Optional):
- Description: English translations of category titles
- Accepted Values: Text, max 256 characters
- Example: Beverages

SubcategoryTitlePt (Column C) (Optional):
- Description: Subcategory names in Portuguese
- Accepted Values: Text, max 256 characters or blank
- Example: Sucos

SubcategoryTitleEn (Column D) (Optional):
- Description: English translations of subcategory titles
- Accepted Values: Text, max 256 characters or blank
- Example: Juices

ItemNamePt (Column E):
- Description: Item names in Portuguese
- Accepted Values: Text, max 256 characters
- Example: Água Mineral

ItemNameEn (Column F) (Optional):
- Description: English translations of item names
- Accepted Values: Text, max 256 characters or blank
- Example: Mineral Water

ItemPrice (Column G):
- Description: Price of each item
- Accepted Values: Text (e.g., 2.50 or 2,50)
- Example: 2.50

Calories (Column H) (Optional):
- Description: The caloric content of the item
- Accepted Values: Numeric
- Example: 150

PortionSize (Column I):
- Description: Portion size for each item in units
- Accepted Values: Text
- Example: 500ml,1,2-3

Availability (Column J) (Optional):
- Description: Current availability of the item
- Accepted Values: Numeric (1 = Yes, 0 = No)
- Example: 1

ItemDescriptionPt (Column K) (Optional):
- Description: Detailed item descriptions in Portuguese
- Accepted Values: Text, max 500 characters
- Example: Contém minerais essenciais

ItemDescriptionEn (Column L) (Optional):
- Description: Detailed item descriptions in English
- Accepted Values: Text, max 500 characters
- Example: Contains essential minerals

Notes:
- It is crucial to ensure all data entered follows the specified formats to maintain the integrity of the database.
- Review the data for accuracy and consistency before submitting the Excel sheet to avoid errors in menu representation.

"""

In [52]:
#get the directory
os.chdir(directory)
IMAGE_DIR = directory

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

#process images in the directory
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
image_files

['DimSum Amoreiras 1.PNG',
 'DimSum Amoreiras 2.PNG',
 'DimSum Amoreiras 3.PNG',
 'DimSum Amoreiras 4.PNG',
 'DimSum Amoreiras 5.PNG']

In [ ]:
# Prompt the user for the excel file name
new_excel_file_name = input("Enter the new Excel file name (without extension): ")
EXCEL_PATH = os.path.join(directory, f"{new_excel_file_name}.xlsx")

#Create the pandas dataframe
df = pd.DataFrame(columns=['CategoryTitlePt', 'CategoryTitleEn', 'SubcategoryTitlePt', 'SubcategoryTitleEn',
                           'ItemNamePt', 'ItemNameEn', 'ItemPrice', 'Calories', 'PortionSize', 'Availability',
                           'ItemDescriptionPt', 'ItemDescriptionEn'])
for image in image_files:
  # Retrieve and encode the image
  image_path = os.path.join(IMAGE_DIR, image)
  image_data = encode_image(image_path)

  #Adding a flag for the headers
  headers_added = False


  # Use GPT-4o to analyze and convert the image
  response = client.chat.completions.create(
      model=Model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": [
              {'type': 'text',
              'text': "Convert this menu image to a structured Excel Sheet Format."},
              {'type': 'image_url',
              'image_url': {'url': f'data:image/png;base64,{image_data}'}}
          ]}],
      temperature = 0
  )

  for row in response.choices[0].message.content.split('\n'):
    if row.startswith('|') and not row.startswith('|-'):  # Ensure that the data is a row and not a header format
        columns = [col.strip() for col in row.split('|')[1:-1]]  # Extract columns from markdown-style table row

        if len(columns) == len(df.columns):  # Ensure row length matches DataFrame columns
            if 'CategoryTitlePt' in columns:  # Check for the header row
                headers_added = True
                continue  # Skip this row

            if headers_added:  # Only process rows after header is detected
                new_row = pd.Series(columns, index=df.columns)
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"Skipping row: {row}")  # Debugging for inconsistent row lengths

In [54]:
print(response.choices[0].message.content.split('\n'))




["Here's the structured Excel sheet format based on the menu image:", '', '| CategoryTitlePt | CategoryTitleEn | SubcategoryTitlePt | SubcategoryTitleEn | ItemNamePt                      | ItemNameEn                      | ItemPrice | Calories | PortionSize | Availability | ItemDescriptionPt | ItemDescriptionEn |', '|-----------------|-----------------|--------------------|--------------------|--------------------------------|--------------------------------|-----------|----------|-------------|--------------|------------------|------------------|', '| Bebidas         | Beverages       | Chás Frios         | Cold Teas          | Branco Tangerina e Menta       | White Tea, Mandarin, Mint      | 3.00      |          |             |              |                  |                  |', '| Bebidas         | Beverages       | Chás Frios         | Cold Teas          | Verde Gengibre e Limão         | Green Tea, Ginger, Lemon       | 3.00      |          |             |              |       

In [56]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
df

,CategoryTitlePt,CategoryTitleEn,SubcategoryTitlePt,SubcategoryTitleEn,ItemNamePt,ItemNameEn,ItemPrice,Calories,PortionSize,Availability,ItemDescriptionPt,ItemDescriptionEn
0,Sopas,Soups,,,Sopa Won Ton,Won Ton Soup,"3,95",,1 unid,1,"Paksoy, Porco e Camarão","Paksoy, Pork & Shrimp"
1,Sopas,Soups,,,Sopa Vegetariana,Vegetarian Soup,"3,95",,1 unid,1,"Paksoy, Bambu, Cogumelo e Verduras","Paksoy, Bamboo, Mushroom & Vegetables"
2,Arroz & Verduras,Rice & Vegetables,,,Arroz branco,Steamed Rice,"1,95",,1 unid,1,Arroz ao vapor,Steamed Rice
3,Arroz & Verduras,Rice & Vegetables,,,Paksoy ao vapor,Steamed Paksoy,"2,40",,1 unid,1,Paksoy ao vapor com Coentros,Steamed Paksoy with Coriander
4,Siao Long Pao,,,,Siao Long Pao Tradicional,Traditional Siao Long Pao,"4,50",,3 unid,1,"Porco, Paksoy e Cogumelo Shiitake","Pork, Paksoy & Shiitake Mushroom"
5,Siao Long Pao,,,,Siao Long Pao Negro,Black Siao Long Pao,"4,50",,3 unid,1,"Porco, Gengibre e Cebolinho","Pork, Ginger & Chives"
6,Gyozas,,,,Gyoza Vegetais,Vegetable Gyoza,"4,50",,3 unid,1,Veg,Veg
7,Gyozas,,,,Gyoza Camarão,Shrimp Gyoza,"4,50",,3 unid,1,Shrimp,Shrimp
8,Gyozas,,,,Gyoza Pato,Duck Gyoza,"4,50",,3 unid,1,Duck,Duck
9,Gyozas,,,,Gyoza Galinha,Chicken Gyoza,"4,50",,3 unid,1,Chicken,Chicken


In [ ]:
df.to_excel(EXCEL_PATH, index=False)
print(f"Excel file saved at: {EXCEL_PATH}")